In [1]:
from dotenv import load_dotenv
load_dotenv('/Users/mdmehranabul/Desktop/Learning/Langchain_Ollama/.env')

True

In [2]:
from langchain_core.prompts import (SystemMessagePromptTemplate, 
                                    HumanMessagePromptTemplate,
                                    PromptTemplate,
                                    ChatMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_ollama import ChatOllama

base_url="http://localhost:11434"
model='llama3.2:3b'


llm=ChatOllama(base_url=base_url,model=model)
llm

ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [3]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

In [4]:
class Joke(BaseModel):
    """Joke to tell the user."""
    setup: str = Field(description = "the setup of the joke")
    punchline : str = Field(description = "the punchline of the joke")
    rating : Optional[int] = Field(description = "the rating of the joke from 1 to 10",default=None)

In [5]:
parser=PydanticOutputParser(pydantic_object=Joke)

In [6]:
instruction = parser.get_format_instructions()

In [7]:
print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke to tell the user.", "properties": {"setup": {"description": "the setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "the punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "the rating of the joke from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [8]:
prompt=PromptTemplate(
    template='''
    Answer the user query with a joke. Here is your formatting instructions. {format_instruction}

    Query: {query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}

)

chain = prompt | llm

In [9]:
output= chain.invoke({'query':'tell me a joke about the cat'})

In [10]:
print(output.content)

{"setup": "Why did the cat join a band?", "punchline": "Because it wanted to be the purr-cussionist!", "rating": 8}


In [15]:
chain = prompt | llm | parser
output= chain.invoke({'query':'tell me a joke about an elephant'})
print(output)

setup='Why did the elephant quit the circus?' punchline='Because it was tired of working for peanuts!' rating=8


### Parsing with .with_structured_output()

In [16]:
llm

ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [18]:
output=llm.invoke('Tell me a joke about a cat')
print(output.content)

Why did the cat join a band?

Because it wanted to be the purr-cussionist.


In [19]:
structured_llm=llm.with_structured_output(Joke)

In [21]:
output=structured_llm.invoke('Tell me a joke about a cat')
print(output)

setup='Why did the cat...' punchline='Why did the cat join a band? Because it wanted to be a purr-cussionist.' rating=8


### JSON output parser

In [25]:
from langchain_core.output_parsers import JsonOutputParser
parser=JsonOutputParser(pydantic_object=Joke)

In [26]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Joke to tell the user.", "properties": {"setup": {"description": "the setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "the punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "the rating of the joke from 1 to 10", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [27]:
prompt=PromptTemplate(
    template='''
    Answer the user query with a joke. Here is your formatting instructions. {format_instruction}

    Query: {query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}

)

chain = prompt | llm
output = chain.invoke({'query':'tell me a joke about the cat'})
print(output.content)

{"setup": "Why did the cat join a band?", "punchline": "Because it wanted to be the purr-cussionist!", "rating": 8}


In [29]:
chain = prompt | llm | parser
output = chain.invoke({'query':'tell me a joke about the cat'})
print(output)

{'setup': 'Why did the cat join a band?', 'punchline': 'Because it wanted to be the purr-cussionist!', 'rating': 8}


### CSV Output Parser

In [30]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser=CommaSeparatedListOutputParser()

print(parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [33]:
format_instruction=parser.get_format_instructions()

prompt=PromptTemplate(
    template='''
    Answer the user query with a list of values. Here is your formatting instructions. 
    {format_instruction}
    Query: {query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'format_instruction': format_instruction}
)

In [35]:
chain = prompt | llm | parser
output = chain.invoke({'query':'generate my website seo keywords. I have content about NLP and LLM.'})
print(output)

['nlp', 'llm', 'artificial intelligence', 'machine learning', 'natural language processing', 'language models', 'deep learning', 'text analysis', 'sentiment analysis', 'chatbots', 'conversational ai']


### Datetime Output Parser

In [36]:
from langchain.output_parsers import DatetimeOutputParser
parser=DatetimeOutputParser()

In [37]:
format_instruction=parser.get_format_instructions()
print(format_instruction)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1436-08-11T12:43:01.461609Z, 1762-02-12T01:36:07.975021Z, 0235-11-22T16:16:30.919100Z

Return ONLY this string, no other words!


In [40]:
prompt=PromptTemplate(
    template='''
    Answer the user query with a datetime. Here is the formatting instruction.
    {format_instruction}
    Query:{query}
    Answer:''',
    input_variables=['query'],
    partial_variables={'format_instruction': format_instruction}
)

In [41]:
chain = prompt | llm | parser

In [47]:
output=chain.invoke({'query':'When was America discovered?'})
print(output)

1492-08-10 04:00:00
